# **Алгоритм индивидуальных рекомендаций фильмов (Recommendation system)**

## **Используемые библиотеки**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import coo_matrix, csr_matrix

import optuna
import optuna.visualization as vis

from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import BaseMethod
from cornac.models import ItemKNN, SVD, MF, NMF, PMF, MMMF, MTER, CTR
from cornac.metrics import MAP, NDCG, RMSE, MAE, Precision
from cornac.experiment import Experiment, Result

from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

np.random.seed(17)

## **Работа с данными**

Данные содержат два файла: **movies** и **ratings**.  
  
Movies содержит информацию о более чем 62.000 фильмов. Для фильмов известны название, год выпуска и жанры.  
  
Ratings - сводная таблица рейтингов (от 0.5 до 5.0), каждому из которых соответствуют ID пользователя и фильма. Всего более 25.000.000 оценок.  

Ссылка на датасет: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system

Загрузим данные и проведем первичный анализ:

In [ ]:
movies = pd.read_csv(r"C:\Users\User\Desktop\data\recommendation\movies.csv")
ratings = pd.read_csv(r"C:\Users\User\Desktop\data\recommendation\ratings.csv")

### **Первичный аналих данных и их обработка**

In [ ]:
print(f'Количество фильмов: {movies.shape[0]}')
movies.head()

In [ ]:
movies.info()

In [ ]:
movies.describe(include='all').T

In [ ]:
print(f'Количество рейтингов: {ratings.shape[0]}')
ratings.head()

Сразу переведем колонку **timestamp** в формат **datetime**, также снизим размер данных (изменим типы на 32-битные), отсортируем рейтинги от старых к новым.  
  
В процессе выполнения проекта выяснилось, что такой объем данных требует слишком больших вычислительных мощностей, поэтому было принято решение о сокращении данных до чуть более чем 2.500.000 записей (учитываются только рейтинги начиная с 2018 года).

In [ ]:
ratings['movieId'] = ratings['movieId'].astype('int32')
ratings['rating'] = ratings['rating'].astype('float32')
ratings['userId'] = ratings['userId'].astype('int32')

ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

ratings = ratings[ratings['timestamp'].dt.year > 2017]

ratings = ratings.sort_values(by='timestamp')
ratings

In [ ]:
ratings.info(show_counts=True)

In [ ]:
ratings.describe(exclude='datetime64').T

In [ ]:
for i in range(0, 6):
  print(f'Количество фильмов, у которых оценок: {i} - {len(np.where(ratings['movieId'].value_counts() == i)[0])}')

print(f'Количество фильмов, имеющих хотя бы одну оценку: {ratings.drop_duplicates('movieId').shape[0]}')

Обработаем колонки **title** и **genres** файла **movies**:  
**title** очистим от лишних символов, разделим на две колонки, отделив год от названия (получим колонки **title** и **year**);  
**genres** преобразуем в список, выведем все уникальные жанры.

In [ ]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

In [ ]:
movies['title'] = movies['title'].apply(clean_title)

title_year = movies['title'].str.rsplit(' ', n=1, expand=True)
title_year.columns = ['title', 'year']
movies.drop(columns='title', inplace=True)

movies = pd.concat([movies, title_year], axis=1)
movies = movies[['movieId', 'title', 'genres', 'year']]

In [ ]:
movies['genres'] = movies['genres'].str.split('|')
unique_genres = pd.Series([genre for genres_list in movies['genres'] for genre in genres_list]).unique()
print(f'Жанры фильмов: {(unique_genres)}')

Очистим **movies** от повторов и пропусков:

In [ ]:
movies = movies.drop_duplicates(subset='title')
movies = movies.dropna()
movies['year'].value_counts()

Заметим, что в колонке **year** для некоторых фильмов оказались слова, а не числа. Видимо, для этих фильмов год изначально не был указан, поэтому разделение произошло по словам.  
  
Удалим эти фильмы из датафрейма, а вместе с ними и просто редковстречающиеся:

In [ ]:
threshold = 7
counts = movies['year'].value_counts()
valid = counts[counts >= threshold].index

movies = movies[movies["year"].isin(valid)]

movies = movies.drop(movies.index[np.where(movies['year'] == '')])
movies = movies[movies['movieId'].isin(ratings['movieId'])]
movies

Оставим только рейтинги, относящиеся к фильмам, оставшимся в **movies**:

In [ ]:
ratings = ratings[ratings['movieId'].isin(movies['movieId'])]
print(f'Количество фильмов в ratings: {len(ratings['movieId'].value_counts())}')
print(f'Количество фильмов в movies: {movies.shape[0]}')
ratings

### **Визуализация данных**

In [ ]:
genre_counts = movies['year'].value_counts()

print(f'Двадцать годов, в которые вышло наибольшее количество фильмов: \n {list(genre_counts.index[:20])}')
print(f'Двадцать годов, в которые вышло наименьшее количество фильмов: \n {list(genre_counts.index[-20:])}')

plt.figure(figsize=(18, 7))
genre_counts.plot(kind='bar')
plt.title('Рапределение фильмов по годам', fontsize=14)
plt.xlabel('Год', fontsize=12)
plt.ylabel('Количество фильмов', fontsize=10)
plt.xticks(ha='right')
plt.show()

In [ ]:
genre_counts = pd.Series([genre for genres_list in movies['genres'] for genre in genres_list]).value_counts()

plt.figure(figsize=(14, 7))
genre_counts.plot(kind='bar')
plt.title('Рапределение фильмов по жанрам', fontsize=14)
plt.xlabel('Жанр', fontsize=12)
plt.ylabel('Количество фильмов', fontsize=12)
plt.xticks(ha='right')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='rating', data=ratings,
             order=sorted(ratings['rating'].unique()))
plt.title('Распределение рейтинга', fontsize=14)
plt.xlabel('Рейтинг', fontsize=12)
plt.ylabel('Частота', fontsize=12)
plt.show()

In [ ]:
sns.boxplot(x="rating", data=ratings)

plt.title('Распределение рейтинга (boxplot)', fontsize=14)
plt.xlabel('Рейтинг', fontsize=12)
plt.show()

In [ ]:
print(f'Среднее количество оценок на пользователя: {np.mean(ratings['userId'].value_counts())}')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

user_counts = np.log(ratings['userId'].value_counts())
user_counts.index = range(0, len(user_counts))

user_counts_min = ratings['userId'].value_counts()[-5000:]
user_counts_min.index = range(0, len(user_counts_min))

user_counts.plot(kind='line', ax=axes[0])
axes[0].set_title('Активность пользователей', fontsize=14)
axes[0].set_xlabel('Номер пользователя', fontsize=12)
axes[0].set_ylabel('Количество оценок (логарифм)', fontsize=12)

user_counts_min.plot(kind='line', ax=axes[1])
axes[1].set_title('5000 самых неактивных пользователей', fontsize=14)
axes[1].set_xlabel('Номер пользователя', fontsize=12)
axes[1].set_ylabel('Количество оценок', fontsize=12)

plt.show()

In [ ]:
print(f'Среднее количество оценок на фильм: {np.mean(ratings['movieId'].value_counts())}')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

movie_counts_max = np.log(ratings['movieId'].value_counts())
movie_counts_max.index = range(0, len(movie_counts_max))

movie_counts_min = ratings['movieId'].value_counts()[-30000:]
movie_counts_min.index = range(0, len(movie_counts_min))

movie_counts_max.plot(kind='line', ax=axes[0])
axes[0].set_title('Популярность фильмов', fontsize=14)
axes[0].set_xlabel('Номер фильма', fontsize=12)
axes[0].set_ylabel('Количество оценок (логарифм)', fontsize=12)

movie_counts_min.plot(kind='line', ax=axes[1])
axes[1].set_title('30000 самых непопулярных фильмов', fontsize=14)
axes[1].set_xlabel('Номер фильма', fontsize=12)
axes[1].set_ylabel('Количество оценок', fontsize=12)

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

user_counts = ratings['userId'].value_counts()
user_counts.index = range(0, len(user_counts))

movie_counts = ratings['movieId'].value_counts()
movie_counts.index = range(0, len(movie_counts_max))

sns.boxplot(x=user_counts, showfliers=False, ax=axes[0])
axes[0].set_title('Активность пользователей', fontsize=14)
axes[0].set_xlabel('Количество оценок пользователей', fontsize=12)

sns.boxplot(x=movie_counts, showfliers=False, ax=axes[1])
axes[1].set_title('Популярность фильмов', fontsize=14)
axes[1].set_xlabel('Количество оценок фильмов', fontsize=12)

plt.show()

In [ ]:
df_time = ratings.set_index('timestamp').resample('ME').size()

plt.figure(figsize=(12,4))
df_time.plot()
plt.title("Количество рейтингов по месяцам", fontsize=14)
plt.ylabel("Количество рейтингов в месяц", fontsize=12)
plt.xlabel("Время", fontsize=12)
plt.show()


Удалим фильмы меньше чем с 6 оценками, а также пользователей, поставивших оценки менее 26 раз:

In [ ]:
ratings = ratings.groupby('movieId').filter(lambda count : len(count)>5)
ratings = ratings.groupby('userId').filter(lambda count : len(count)>25)

## **Подготовка данных**

Разделим данные **без перемешивания**, чтобы модели учились предсказывать будущие рейтинги на основе уже известных. Такое разбиение имитирует сценарий реальных предсказаний для пользователей.

In [ ]:
ratings_train, ratings_test = train_test_split(ratings.drop(columns='timestamp'),
                                               test_size=0.2, shuffle=False)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

user_test_counts = ratings_test['userId'].value_counts()
user_test_counts.index = range(0, len(user_test_counts))

movie_test_counts = ratings_test['movieId'].value_counts()
movie_test_counts.index = range(0, len(movie_test_counts))

user_test_counts.plot(kind='line', ax=axes[0])
axes[0].set_title('Активность пользователей тестовой выборки', fontsize=14)
axes[0].set_xlabel('Номер пользователя', fontsize=12)
axes[0].set_ylabel('Количество оценок', fontsize=12)

movie_test_counts.plot(kind='line', ax=axes[1])
axes[1].set_title('Популярность фильмов тестовой выборки', fontsize=14)
axes[1].set_xlabel('Номер фильма', fontsize=12)
axes[1].set_ylabel('Количество оценок', fontsize=12)

plt.show()

## **Обучение моделей**

### **Experiment из Cornac**

Запустим Эксперимент из библиотеки **Cornac**. Это сравнительный фреймворк для мультимодальных рекомендательных систем, позволяющий быстро проводить эксперименты и легко реализовывать новые модели.  
  
В качестве моделей инициализируем **ItemKNN, Matrix Factorization, Probabilistic Matrix Factorization, SVD, Non-negative Matrix Factorization, Maximum Margin Matrix Factorization**.
  
Все методы, кроме первого, основываются на **матричной факторизации** (метод разложения большой матрицы на произведение двух или более матриц меньшего размера, чтобы выявить скрытые закономерности и латентные факторы, что позволяет предсказывать пропущенные значения и делать точные рекомендации.)

In [ ]:
evaluation = BaseMethod.from_splits(train_data=ratings_train.values,
                                        test_data=ratings_test.values,
                                        rating_threshold=4.0,
                                        exclude_unknowns=True)


models = [ItemKNN(k=20, similarity='cosine', num_threads=8), #Item-based KNN

          MF(k=50, max_iter=100, learning_rate=0.01, batch_size=256, #Matrix Factorization
             lambda_reg=0.01, use_bias=True, early_stop=True, num_threads=8),

          PMF(k=30, max_iter=100, learning_rate=0.001, gamma=0.9, #Probabilistic Matrix Factorization
              lambda_reg=0.005),

          SVD(k=50, max_iter=100, learning_rate=0.01, lambda_reg=0.01, #Matrix Factorization with biases
              early_stop=True, num_threads=8),

          NMF(k=50, max_iter=100, learning_rate=0.005, lambda_u=0.06, #Non-negative Matrix Factorization
              lambda_v=0.06, lambda_bu=0.02, lambda_bi=0.02,
              use_bias=False, num_threads=8),

          MMMF(k=50, max_iter=100, learning_rate=0.001, lambda_reg=0.01, #Maximum Margin Matrix Factorization
               num_threads=8)]

Возьмем метрики **MAP, NDCG, Precision_at_k**, которые показывают правильность ранжирования и вхождение правильных объектов в первую десятку (в нашем случае) рекоммендаций, а также **RMSE и MAE**, показывающие отклонение предсказаний оценок от истинного рейтинга:

In [ ]:
metrics = [MAP(), NDCG(k=10), Precision(k=10), RMSE(), MAE()]

In [ ]:
Experiment(eval_method=evaluation, models=models,
           metrics=metrics, user_based=True, verbose=True).run()

Из результатов Эксперимента видно, что лучший результат по метрикам ранжирования у модели **Maximum Margin Matrix Factorization**.  
  
Также эта модель показала неплохую скорость и очень большие ошибки **RMSE** и **MAE**. Это связано с тем, что данная модель в первую очередь рассчитана на ранжирование, а не на точное предсказание рейтингов.

Оптимизируем эту модель по основным параметрам по целевой метрике **Precision@K**:

In [ ]:
def objective_mf(trial):
  params = {'k' : trial.suggest_int("k", 20, 200),
            'lambda_reg' : trial.suggest_float(" lambda_reg", 1e-5, 1e-1, log=True),
            'learning_rate' : trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
            'max_iter' : trial.suggest_int("max_iter", 50, 200)}

  evaluation_search = BaseMethod.from_splits(train_data=ratings_train.values,
                                        test_data=ratings_test.values,
                                        rating_threshold=4.0,
                                        exclude_unknowns=True)
  model_search = MMMF(**params, num_threads=8)
  metric_search = [Precision(k=10)]

  results_search = evaluation_search.evaluate(model=model_search,
                                       metrics=metric_search, user_based=True)
  test_result, _ = results_search
  score_prec = test_result.metric_avg_results["Precision@10"]

  return score_prec

study_mf = optuna.create_study(direction="maximize")
study_mf.optimize(objective_mf, n_trials=100)

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(study_mf)
optuna.visualization.matplotlib.plot_optimization_history(study_mf)
print(f'Лучшие параметры модели: {study_mf.best_params}')

В результате получим лучшие параметры для данной модели, а также самое высокое значение целевой метрики - 0.033.

### **LightFM**

Подготовим данные для модели **LightFM** (библиотека для создания рекомендательных систем, которая сочетает в себе коллаборативную фильтрацию с контентной):

In [ ]:
ratings_test = ratings_test[ratings_test['userId'].isin(ratings_train['userId']) &
                            ratings_test['movieId'].isin(ratings_train['movieId'])]

In [ ]:
dataset = Dataset()

dataset.fit(users=ratings_train['userId'].unique(),
            items=ratings_train['movieId'].unique())

(interactions_train, weights_train) = dataset.build_interactions(
    (u, i, l) for u, i, l in ratings_train[["userId", "movieId", "rating"]].values)

(interactions_test, weights_test) = dataset.build_interactions(
    (u, i, l) for u, i, l in ratings_test[["userId", "movieId", "rating"]].values)

Обучим первую модель со стандартными значениями гиперпараметров:

In [ ]:
model = LightFM(no_components=30, loss='warp')

model.fit(interactions=interactions_train,
          sample_weight=weights_train,
          epochs=50, num_threads=8, verbose=True)

In [ ]:
light_fm_prec = np.mean(precision_at_k(model=model,
                                       test_interactions=interactions_test,
                                       k=10, num_threads=8))

print(f'Precision@K = {light_fm_prec}')

Подберем гиперпараметры для этой модели:

In [ ]:
def objective_lightfm(trial):
  params = {'no_components' : trial.suggest_int("no_components", 10, 200),
            'learning_schedule' : trial.suggest_categorical('learning_schedule', ['adagrad', 'adadelta']),
            'loss' : trial.suggest_categorical('loss', ['bpr', 'warp']),
            'learning_rate' : trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
            'max_sampled' : trial.suggest_int("max_sampled", 5, 50)}


  lightfm_model = LightFM(**params)

  lightfm_model.fit(interactions=interactions_train, sample_weight=weights_train,
                    epochs=10, num_threads=8)

  score = precision_at_k(model=lightfm_model,
                          test_interactions=interactions_test, k=10,
                          num_threads=8).mean()
  return score

study_lightfm = optuna.create_study(direction="maximize")
study_lightfm.optimize(objective_lightfm, n_trials=100, show_progress_bar=True)

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(study_lightfm)
optuna.visualization.matplotlib.plot_optimization_history(study_lightfm)
print(f'Лучшие параметры модели: {study_lightfm.best_params}')

Результат Precision@K = 0.0337.  

## **Сравнение результатов**

Сравним результаты двух оптимизированных моделей:

In [ ]:
print(f'LightFM: {study_lightfm.best_value}')
print(f'MMMF: {study_mf.best_value}')

В итоге модель LightFM оказалось незначительно лучше. Можно говорить об идентичных результатах

##**Дальнейшие действия**

1). Использовать данные за все года, а не только начиная с 2018 года;  
2). Использовать более продвинутые модели, основанные на глубоком обучении;  
3). Использовать контентные признаки (год выпуска фильмов, жанры и тд);  
4). Менее агрессивно фильтровать данные, оставив большее количество фильмов и оценок.  
  
Все эти действия сопряжены с использованием большого количества вычислительных ресурсов.

